# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
F. Walter  ->  F. Walter  |  ['F. Walter']
T. Henning  ->  T. Henning  |  ['T. Henning']


G. Rouillé  ->  G. Rouillé  |  ['G. Rouillé']
Arxiv has 60 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2506.14921
extracting tarball to tmp_2506.14921...

 done.


Found 117 bibliographic references in tmp_2506.14921/aa54144-25.bbl.
Retrieving document from  https://arxiv.org/e-print/2506.15039
extracting tarball to tmp_2506.15039...

 done.
Retrieving document from  https://arxiv.org/e-print/2506.15286
extracting tarball to tmp_2506.15286...

...

✔ → 15263:\section{MIRI-MRS observations}\label{sec:observations}
  ↳ 24265:\section{Results}\label{sec:results}


✔ → 24265:\section{Results}\label{sec:results}
  ↳ 59144:\section{Discussion and summary}\label{sec:discussion}


✔ → 59144:\section{Discussion and summary}\label{sec:discussion}
  ↳ 70336:\begin{appendix}


✔ → 70336:\begin{appendix}
  ↳ 76815:end
F. Walter  ->  F. Walter  |  ['F. Walter']
T. Henning  ->  T. Henning  |  ['T. Henning']


Found 85 bibliographic references in tmp_2506.15286/paperCenA.bbl.
Issues with the citations
syntax error in line 18: '=' expected
Retrieving document from  https://arxiv.org/e-print/2506.15550
extracting tarball to tmp_2506.15550... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.14921-b31b1b.svg)](https://arxiv.org/abs/2506.14921) | **The MUSE view of the Sculptor galaxy: survey overview and the planetary nebulae luminosity function**  |
|| E. Congiu, et al. -- incl., <mark>K. Kreckel</mark>, <mark>E. Schinnerer</mark> |
|*Appeared on*| *2025-06-20*|
|*Comments*| *21 pages, 17 figures, accepted for publication in A&A*|
|**Abstract**|            NGC 253, the Sculptor galaxy, is the southern, massive, star-forming disk galaxy closest to the Milky Way. In this work, we present a new 103-pointing MUSE mosaic of this galaxy covering the majority of its star-forming disk up to 0.75xR25. With an area of ~20x5 arcmin2 (~20x5 kpc2, projected) and a physical resolution of ~15 pc, this mosaic constitutes one of the largest, highest physical resolution integral field spectroscopy surveys of any star-forming galaxy to date. Here, we exploit the mosaic to identify a sample of ~500 planetary nebulae (~20 times larger than in previous studies) to build the planetary nebula luminosity function (PNLF) and obtain a new estimate of the distance to NGC 253. The value obtained is 17% higher than estimates returned by other reliable measurements, mainly obtained via the top of the red giant branch method (TRGB). The PNLF also varies between the centre (r < 4 kpc) and the disk of the galaxy. The distance derived from the PNLF of the outer disk is comparable to that of the full sample, while the PNLF of the centre returns a distance ~0.9 Mpc larger. Our analysis suggests that extinction related to the dust-rich interstellar medium and edge-on view of the galaxy (the average E(B-V) across the disk is ~0.35 mag) plays a major role in explaining both the larger distance recovered from the full PNLF and the difference between the PNLFs in the centre and in the disk.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.15286-b31b1b.svg)](https://arxiv.org/abs/2506.15286) | **MICONIC: JWST/MIRI MRS reveals a fast ionized gas outflow in the central region of Centaurus A**  |
|| A. Alonso-Herrero, et al. -- incl., <mark>F. Walter</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2025-06-20*|
|*Comments*| *Accepted for publication in Astronomy & Astrophysics*|
|**Abstract**|            We present a kinematical study of the ionized and molecular gas in the central region (~7-14"~100-200pc) of the nearby radio galaxy Cen A. We used JWST/MIRI MRS 5-28$\mu$m observations taken as part of the MIR Characterization of Nearby Iconic galaxy Centers (MICONIC) of the MIRI EC. The two gas phases present contrasting morphologies and kinematics. The brightest emission from the ionized gas, traced with a range of IP lines ([Fe II] to [Ne VI]), is extended along the direction of the radio jet. We also detected emission from low IP emission lines and H$_2$ transitions in the galaxy disk. Both gas phases present rotational motions but also complex kinematics. The observations reveal several ionized gas kinematical features that are consistent with simulation predictions of a jet-driven bubble and outflow interacting with the galaxy ISM. These include broad components in the nuclear line profiles ($\sigma$~600km/s), high velocities (~ +1000, -1400km/s) confined within the nuclear region, velocities of hundreds of km/s in several directions in the central 2", and enhanced velocity dispersions perpendicular to the radio jet. Moreover, we find evidence of shock excitation in the nuclear region based on MIR line ratios. We compared the ionized gas mass outflow rate with Cen A's AGN luminosity and radio jet power and demonstrate that both mechanisms provide sufficient energy to launch the outflow. The noncircular motions observed in the H$_2$ lines can be reproduced with either a warped rotating disk model or a radial component. The latter might be to related to gas streamers detected in cold molecular gas. There is no clear indication of a fast nuclear H$_2$ outflow, only a weak blueshifted component. This could be due to a relatively low nuclear warm H$_2$ column density and/or the limited geometrical coupling of Cen A's inner radio jet with the circumnuclear disk of the galaxy. (Abridged)         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.15039-b31b1b.svg)](https://arxiv.org/abs/2506.15039) | **Unveiling the Cosmic Dance of Repeated Nuclear Transient ASASSN-14ko: Insights from Multiwavelength Observations**  |
|| S. Huang, et al. -- incl., <mark>Y. Wang</mark>, <mark>Y. Wang</mark>, <mark>Y. Wang</mark> |
|*Appeared on*| *2025-06-20*|
|*Comments*| *Accepted for publication in ApJ, 16 pages, 10 figures*|
|**Abstract**|            ASASSN-14ko is a periodically repeating nuclear transient. We conducted high-cadence, multiwavelength observations of this source, revealing several recurrent early bumps and rebrightenings in its UV/optical light curves. The energy released during these bumps and rebrightenings shows a diminishing trend in recent UV/optical outbursts, which we monitored through multiwavelength observations. These features can be ascribed to the interaction between stream debris and the expanded disk in the repeated partial tidal disruption event. The X-ray light curve exhibits an inverse pattern compared to the UV/optical bands, displaying sporadic outbursts. Furthermore, our observations demonstrate that the blackbody temperature and radius in each outburst increase with the UV/optical luminosity, and such evolution resembles that observed in X-ray quasiperiodic eruptions, whereas distinguishing it from typical tidal disruption events.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.15550-b31b1b.svg)](https://arxiv.org/abs/2506.15550) | **Ultraviolet Spectra of Comets: Rejecting the Detection of Pentacene, Toluene, and Fe+**  |
|| <mark>G. Rouillé</mark> |
|*Appeared on*| *2025-06-20*|
|*Comments*| *Accepted for publication in Journal of Astrophysics and Astronomy, 11 pages, 9 figures*|
|**Abstract**|            A recent study announced the detection of three bands in the ultraviolet emission spectra of more than a dozen comets, assigning two of them to pentacene (C$_{22}$H$_{14}$) and the third one to toluene (C$_7$H$_8$). The comparison of the spectra with the results of exploitable laboratorymeasurements on rare-gas-matrix-isolated pentacene and on jet-cooled toluene does not reveal elements that would justify the assignment, which is therefore unsubstantiated. The study also claimed the detection of an Fe II line in the gas of all but one comet. Yet, spectroscopic data on Fe II do not corroborate the attribution. Because spectroscopic measurements on the ultraviolet emission of pentacene in the gas phase are not available, this work also presents a synthetic spectrum of the S$_5$ $\rightarrow$ S$_0$ transition relevant to the wavelength range of the observations. Calculated using density functional theory and its time-dependent extension, the synthetic spectrum may facilitate the search for pentacene fluorescence in cometary spectra until laboratory measurements are accessible.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2506.14921/./fig/NGC253_EBV.png', 'tmp_2506.14921/./fig/Comparison_all.png', 'tmp_2506.14921/./fig/NGC253_FULL.png']
copying  tmp_2506.14921/./fig/NGC253_EBV.png to _build/html/
copying  tmp_2506.14921/./fig/Comparison_all.png to _build/html/
copying  tmp_2506.14921/./fig/NGC253_FULL.png to _build/html/
exported in  _build/html/2506.14921.md
    + _build/html/tmp_2506.14921/./fig/NGC253_EBV.png
    + _build/html/tmp_2506.14921/./fig/Comparison_all.png
    + _build/html/tmp_2506.14921/./fig/NGC253_FULL.png
found figures ['tmp_2506.15286/./figure6a.png', 'tmp_2506.15286/./figure6b.png', 'tmp_2506.15286/./figure6c.png', 'tmp_2506.15286/./figure6d.png', 'tmp_2506.15286/./figure5a.png', 'tmp_2506.15286/./figure5b.png', 'tmp_2506.15286/./figureA3_1.png', 'tmp_2506.15286/./figureA3_2.png', 'tmp_2506.15286/./figureA3_3.png', 'tmp_2506.15286/./figureA3_4.png', 'tmp_2506.15286/./figureA3_5.png']
copying  tmp_2506.15286/./figure6a.png to _build/html/
copying  tmp_2506.15286/./

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Hb}{\hbox{H\beta}}$
$\newcommand{\Ha}{\hbox{H\alpha}}$
$\newcommand{\oi}{[O~\textsc{i}]\lambda6300}$
$\newcommand{\oiii}{[O~\textsc{iii}]\lambda5007}$
$\newcommand{\nii}{[N~\textsc{ii}]\lambda6584}$
$\newcommand{\siia}{[S~\textsc{ii}]\lambda6717}$
$\newcommand{\siib}{[S~\textsc{ii}]\lambda6731}$
$\newcommand{\sii}{[S~\textsc{ii}]\lambda\lambda6717,6731}$
$\newcommand{\siii}{[S~\textsc{iii}]\lambda9068}$
$\newcommand{\hei}{He~\textsc{i}~\lambda5875}$
$\newcommand{\heii}{He~\textsc{ii}~\lambda4685}$
$\newcommand{\oion}{[O~\textsc{i}]}$
$\newcommand{\oiion}{[O~\textsc{ii}]}$
$\newcommand{\oiiion}{[O~\textsc{iii}]}$
$\newcommand{\niion}{[N~\textsc{ii}]}$
$\newcommand{\siiion}{[S~\textsc{iii}]}$
$\newcommand{\siion}{[S~\textsc{ii}]}$
$\newcommand{\heion}{He~\textsc{i}}$
$\newcommand{\heiion}{He~\textsc{ii}}$
$\newcommand{\ngc}{NGC~}$
$\newcommand{\Msun}{\mbox{M}_{\sun}}$
$\newcommand{\Lsun}{\mbox{L}_{\sun}}$
$\newcommand{\hii}{H~{\sc ii}}$
$\newcommand{\sn}{S/N}$
$\newcommand{\ebv}{E(B-V)}$
$\newcommand{\izi}{{\tt IZI}}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.1}$</div>



<div id="title">

# The MUSE view of the Sculptor galaxy: survey overview and the planetary nebulae luminosity function.

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.14921-b31b1b.svg)](https://arxiv.org/abs/2506.14921)<mark>Appeared on: 2025-06-20</mark> -  _21 pages, 17 figures, accepted for publication in A&A_

</div>
<div id="authors">

E. Congiu, et al. -- incl., <mark>K. Kreckel</mark>, <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** $\ngc$ 253, the Sculptor galaxy, is the southern, massive, star-forming disk galaxy closest to the Milky Way.  In this work, we present a new 103-pointing MUSE mosaic of this galaxy covering the majority of its star-forming disk up to $0.75\times$ R $_{25}$ .  With an area of $\sim20\times5 \si{arcmin^2}$ ( $\sim20\times5 \si{kpc^2}$ , projected) and a physical resolution of $\sim 15 \si{pc}$ , this mosaic constitutes one of the largest, highest physical resolution integral field spectroscopy surveys of any star-forming galaxy to date.  Here, we exploit the mosaic to identify a sample of $\sim$ 500 planetary nebulae ( $\sim 20$ times larger than in previous studies) to build the planetary nebula luminosity function (PNLF) and obtain a new estimate of the distance to $\ngc$ 253.  The value obtained is $17\%$ higher than estimates returned by other reliable measurements, mainly obtained via the top of the red giant branch method (TRGB).  The PNLF also varies between the centre (r $< 4 \si{kpc}$ ) and the disk of the galaxy.  The distance derived from the PNLF of the outer disk is comparable to that of the full sample, while the PNLF of the centre returns a distance $\sim$ $\SI{0.9}{Mpc}$ larger.  Our analysis suggests that extinction related to the dust-rich interstellar medium and edge-on view of the galaxy (the average $\ebv$ across the disk is $\sim 0.35 \si{mag}$ ) plays a major role in explaining both the larger distance recovered from the full PNLF and the difference between the PNLFs in the centre and in the disk.

</div>

<div id="div_fig1">

<img src="tmp_2506.14921/./fig/NGC253_EBV.png" alt="Fig17" width="100%"/>

**Figure 17. -** $\ebv$ map of $\ngc$253. The $\ebv$ has been calculated from the Balmer decrement using a convolved (to a \SI{5}{arcsec} FWHM Gaussian PSF) and binned version of the data, in order to detect both $\Ha$ and $\Hb$ across the majority of the FOV. (*fig:ebv*)

</div>
<div id="div_fig2">

<img src="tmp_2506.14921/./fig/Comparison_all.png" alt="Fig4" width="100%"/>

**Figure 4. -** Comparison between our PNe photometry, [Rekola, Richer and McCall (2005)]() and [Jacoby, et. al (2024)](). For [Jacoby, et. al (2024)]() we performed the comparison using both the data reduced by us, and the cubes directly recovered from the ESO archive. Solid symbols represent PNe that were used by the original authors in their PNLF computation. Open symbols represent objects that were not included in the final fit. (*fig:phot_comp*)

</div>
<div id="div_fig3">

<img src="tmp_2506.14921/./fig/NGC253_FULL.png" alt="Fig12" width="100%"/>

**Figure 12. -** Colour images of $\ngc$253 produced by combining broad-band images and emission line maps extracted from the MUSE data cube. The mosaic covers an area of $20\times5 \si{arcmin^2}$ and it includes roughly 9 million independent spectra. The top panel show a composition of three broad-band filters: g-band in blue, r-band in green, and i-band in red (Acknowledgement: ESO/M. Kornmesser). We can see the full structure of the galaxy disk, with the prominent bar and complex dust filaments that follow the distribution of the spiral arms of the galaxy. We can also see the change in colour caused by the presence of the central starburst of the galaxy.
    The bottom panel is a composition of emission line maps with $\oi$ii in blue, $\Ha$ in green and $\sii$ in red.
    This highlights the multitude of ionised gas structures we observe in the galaxy.
    The $\hii$ regions distribution highlights the structure of the spiral arms, while the $\oi$ii and $\sii$ emission clearly show the outline of the outflow. Nebulae with different properties can be identified across the field, like the blue, $\oi$ii emitting PNe, the green $\Ha$ bright $\hii$ regions, and the pink, $\sii$ emitting SNRs. (*fig:ngc253_vri*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.14921"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# MICONIC: JWST/MIRI MRS reveals a fast ionized gas    outflow in the central region of    Centaurus A

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.15286-b31b1b.svg)](https://arxiv.org/abs/2506.15286)<mark>Appeared on: 2025-06-20</mark> -  _Accepted for publication in Astronomy & Astrophysics_

</div>
<div id="authors">

A. A. Herrero, et al. -- incl., <mark>F. Walter</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** We present a kinematical study of the ionized and molecular gas in  the central region ( $\sim$ 7--14 $\arcsec$ $\sim$ 100--200 pc) of the  nearby radio galaxy Centaurus A  (Cen A). We used JWST/MIRI MRS $\sim 5-28 \mu$ m observations taken as part of the Mid-Infrared   Characterization of Nearby Iconic galaxy Centers(MICONIC) of the MIRI EuropeanConsortium. The two gas phases present contrasting morphologiesand kinematics in Cen A. The brightest emission from the ionized gas, tracedwith a range of ionization potential (IP) lines analyzed here (from [ Fe ${\sc ii}$ ] to [ Ne ${\sc vi}$ ] ), is extended along the direction of the radio jet.  We also detectedemission from low IP emission lines and $H_2$ transitions in thegalaxy disk region mapped with MRS. Both gas phases present rotationalmotions in the disk but also complex kinematics. The MRS observationsreveal  several ionized gas kinematical features that are consistent with simulationpredictions of a jet-driven bubble and outflow interacting with the galaxyinterstellar medium. Theseinclude broad components in the nuclear line profiles ( $\sigma\sim600 {\rm km s}^{-1}$ in [ Ar ${\sc ii}$ ] and [ Ne ${\sc iii}$ ] ),  highvelocities (reaching approximately +1000, $-1400 {\rm km s}^{-1}$ )confined within the nuclear region, velocities of hundreds ofkilometers per secondin several directions in the central 2 $\arcsec$ , and enhanced velocitydispersions perpendicular to the radio jet. Moreover, we find evidenceof shock excitation in the nuclear region of Cen Abased on mid-infrared line ratios. We compared the ionized gas mass outflow ratewith Cen A's active galactic nucleus (AGN) luminosity and radio jetpower and demonstrate that bothmechanisms provide sufficient energy to launch the outflow. Thenoncircular motions observed in the mid-infrared $H_2$ lines can bereproduced with either a  warped rotating disk model or a radialcomponent. The latter might be to related to gas streamers detected incold molecular gas. Notably, there is no clear indication of a fast $nuclearH_2$ outflow in Cen A, only a weak blueshifted component in the lineprofiles. This could be due to a relatively low nuclear warm $H_2$ column density and/or the limited geometricalcoupling of Cen A's inner radiojet with the circumnuclear disk of the galaxy.

</div>

<div id="div_fig1">

<img src="tmp_2506.15286/./figure6a.png" alt="Fig16.1" width="25%"/><img src="tmp_2506.15286/./figure6b.png" alt="Fig16.2" width="25%"/><img src="tmp_2506.15286/./figure6c.png" alt="Fig16.3" width="25%"/><img src="tmp_2506.15286/./figure6d.png" alt="Fig16.4" width="25%"/>

**Figure 16. -** Same as Fig. \ref{fig:maintextalucinemapsH2} but for
    [Ar {\sc ii}], [Ne {\sc vi}],
       [Ne {\sc iii}], and [O {\sc iv}](from top to
       bottom). (*fig:maintextalucinemapsFSL*)

</div>
<div id="div_fig2">

<img src="tmp_2506.15286/./figure5a.png" alt="Fig15.1" width="50%"/><img src="tmp_2506.15286/./figure5b.png" alt="Fig15.2" width="50%"/>

**Figure 15. -** Maps of  $H_2$ S(5) at $6.909 \mu$m (top) and
         $H_2$ S(1) at $17.03 \mu$m (bottom)  constructed as
       explained in Sect. \ref{subsec:analysis}.
       Panels show  the intensity and contours in a square root
       scale in arbitrary units (left),
       the mean-velocity field in units of km s$^{-1}$(middle),
       and the velocity dispersion map $\sigma$(corrected for instrumental
       resolution)
       in units of km s$^{-1}$(right). The velocity contours are in a linear
       scale (solid lines positive values and dashed lines negative
       values). The star symbol marks the peak of the continuum
       adjacent to the line, that is, the AGN position. The
       0,0 point on the axes refers to the center of the corresponding sub-channel
       array. North is up and east to the left. (*fig:maintextalucinemapsH2*)

</div>
<div id="div_fig3">

<img src="tmp_2506.15286/./figureA3_1.png" alt="Fig19.1" width="20%"/><img src="tmp_2506.15286/./figureA3_2.png" alt="Fig19.2" width="20%"/><img src="tmp_2506.15286/./figureA3_3.png" alt="Fig19.3" width="20%"/><img src="tmp_2506.15286/./figureA3_4.png" alt="Fig19.4" width="20%"/><img src="tmp_2506.15286/./figureA3_5.png" alt="Fig19.5" width="20%"/>

**Figure 19. -** Same as Fig. \ref{fig:maintextalucinemapsFSL} but for  [Fe {\sc
      ii}] at $5.34 \mu$m, [S {\sc iv}],  [Ne {\sc ii}], [Ne {\sc v}] at $14.32 \mu$m, and
[S {\sc iii}](from top to bottom). (*fig:appendixalucinemaps*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.15286"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

532  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

15  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
